### Exploratory Data Analysis - Summarise Attributes
For each ticker csv file, compile descriptive stats, earliest reported value, % nan values by attribute

In [ ]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import os
import urllib
import json
import requests
from io import StringIO

from pathlib import Path
from eod import EodHistoricalData
#import pandas_datareader as pdr
#import yfinance as yf
#import talib

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

pd.set_option('display.max_rows', None)

In [ ]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'

In [ ]:
api_key = "618f834b7bfe27.18132752"

In [ ]:
client = EodHistoricalData(api_key)

In [ ]:
resp = client.get_exchange_symbols(exchange='NYSE')

In [ ]:
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [ ]:
#iterate through fundamental csv files and create sumarry statistics for EDA and compile for each attribute
for ticker in tickers:
    filepath = Path(path_prefix + path_fun + '\df_fun_{}.csv'.format(ticker))
    print(ticker)
    if os.path.isfile(filepath) == True:
        df_fun = pd.read_csv(filepath, index_col=0).transpose()
        df_fun.drop(['date', 'filing_date', 'currency_symbol'], inplace=True, axis=1)
        df_fun = df_fun.astype(np.float64)
        df_fun.sort_index(ascending=False, inplace=True)
        for i in range(len(df_fun.columns)):
            col = df_fun.columns[i]
            desc = df_fun[col].describe().to_frame()
            earliest_val = df_fun[col].last_valid_index()
            desc.loc['earliest_val', col] = earliest_val
            pct_nan = df_fun[col].loc[:earliest_val].isna().sum() / df_fun[col].loc[:earliest_val].shape[0] * 100
            desc.loc['pct_nan', col] = pct_nan
                
            filepath = Path(path_prefix + path_eda + 'df_eda_{}.csv'.format(col))
            if os.path.isfile(filepath) == False:
                filepath.parent.mkdir(parents=True, exist_ok=True)  
                desc.to_csv(filepath)
            else:  
                df_desc = pd.read_csv(filepath, index_col=0)
                suffix = '_{}'.format(ticker)
                df_desc = df_desc.join(desc, rsuffix=suffix)
                df_desc.to_csv(filepath)

Code to remove overrun....

In [ ]:
os.chdir(r'C:\Users\OEM\tickers\attributes')

In [ ]:
files = [f for f in os.listdir() if os.path.isfile(f)]

In [ ]:
for file in files:
    filepath = Path(r'C:\Users\OEM\tickers\attributes\{}'.format(file))
    df = pd.read_csv(filepath, parse_dates=True, index_col=0, dtype='float64')
    filt = df.columns.str.contains('1')
    mask = df.columns[filt]
    df.drop(mask, axis=1, inplace=True)
    df = df[df.index > dt.datetime(2003, 12, 2)]
    df.to_csv(filepath)